# Importation des packages et connexion au S3

In [1]:
from helpers import *
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
s3=s3_connection()

connection successful


# 1. La motivation

In [3]:
regional_stat=s3.get_tables_from_s3("clichere/diffusion/final_comm.parquet")
print(regional_stat[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

        pop2022  revmoy2022  psup2022  pcadr2022
count  36632.00    34720.00  35222.00   35065.00
mean    1829.92    17049.74      0.27       0.12
std     9954.65     5194.78      0.17       0.13
min        0.00     3350.48      0.00       0.00
25%      182.00    14011.97      0.16       0.00
50%      428.00    16321.86      0.26       0.08
75%     1116.00    19124.01      0.37       0.18
max   875791.00   126903.91      1.00       1.00


In [4]:
df=s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")
print(df[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

        pop2022  revmoy2022  psup2022  pcadr2022
count   2808.00     2808.00   2808.00    2808.00
mean   18078.54    18221.52      0.29       0.14
std    50290.90     6608.60      0.13       0.11
min       55.00     5818.04      0.00       0.00
25%     1570.25    14922.08      0.20       0.07
50%     4067.50    16931.46      0.28       0.12
75%    12203.00    19639.30      0.37       0.19
max   875791.00   107572.81      1.00       0.82


On voit que la population moyenne diminue lorsque l'on sélectionne les communes ayant une gare. Cela correspond au fait que les petites communes ont moins de chance d'avoir une gare. On passe de 36000 communes à 2800 communes, ce qui donne tout de même beaucoup d'observations. On remarque que le revenu moyen d'une commune en France est plus élevé lorsque l'on sélectionne seulement les communes comprenant une gare. De même, les personnes sont en moyenne plus diplômée et appartiennent aux catégories socio-professionnelles plus élevées également. 

On veut montrer qu'il existe un lien entre le revenu, la catégorie socio-professionnelle (csp) et le fait de prendre un train en France. Pour cela, on effectue une régression linéaire avec le nombre total de voyageurs comme variable dépendante. Notre but est d'expliquer pourquoi est-ce que cette variable est différente en fonction des communes, en contrôlant par la population de la commune. Les premières visualisations et les premières données suggèrent qu'il peut y avoir un lien entre son revenu et le fait de prendre un train.

In [24]:
print(df.columns)

Index(['total_voyageurs_2022', 'nom_gare', 'uic', 'codeinsee', 'lon_gare',
       'lat_gare', 'codecommune', 'nomcommune', 'pop2022', 'revmoy2022',
       'pbac2022', 'psup2022', 'pagri2022', 'pindp2022', 'pcadr2022',
       'pempl2022', 'pouvr2022', 'pchom2022', 'dist_closest_station_km',
       'nb_stations_same_commune', 'regions', 'freq_par_hab'],
      dtype='object')


# 2. Régressions

In [5]:
df['pnodip2022']=1-df['pbac2022']
df[['pbac2022', 'psup2022', 'pnodip2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'ppint2022']]=df[['pbac2022', 'psup2022', 'pnodip2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'ppint2022']]*100

In [6]:

X = df[['revmoy2022', 'pop2022', 'pnodip2022', 'psup2022', 'pcadr2022', 'pouvr2022', 'pchom2022', 'pempl2022','ppint2022', 'pagri2022', 'dist_closest_station_km']]
Y = df['total_voyageurs_2022']  

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     total_voyageurs_2022   R-squared:                       0.146
Model:                              OLS   Adj. R-squared:                  0.143
Method:                   Least Squares   F-statistic:                     43.59
Date:                  Thu, 19 Dec 2024   Prob (F-statistic):           3.89e-88
Time:                          16:44:03   Log-Likelihood:                -45803.
No. Observations:                  2808   AIC:                         9.163e+04
Df Residuals:                      2796   BIC:                         9.170e+04
Df Model:                            11                                         
Covariance Type:              nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const 

On enlève des csp les professions indépendantes qui regroupe des personnes très hétérogènes pour éviter la colinéarité parfaite. On obtient des coefficients significatifs sauf pour le taux de personnes sans diplôme dans la commune.

Essayons de passer au logarithme du nombre de voyageurs pour capturer des potentielles non-linéarités.

In [7]:
df['log_nb_voy2022']= np.log(df['total_voyageurs_2022'])

In [9]:
X = df[['revmoy2022', 'pop2022', 'psup2022', 'pcadr2022', 'pouvr2022', 'pchom2022', 'pempl2022', 'pagri2022', 'ppint2022', 'dist_closest_station_km']]
Y = df['log_nb_voy2022']  

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         log_nb_voy2022   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     66.77
Date:                Thu, 19 Dec 2024   Prob (F-statistic):          2.08e-122
Time:                        17:05:50   Log-Likelihood:                -6257.1
No. Observations:                2808   AIC:                         1.254e+04
Df Residuals:                    2797   BIC:                         1.260e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

On note que les coefficients sont significatifs sauf pour le revenu moyen des communes. Après avoir contrôlé pour les autres variables, et en ajustant le nombre de voyageur en logarithme, l'effet du revenu n'est plus significatif. En revanche, tous les autres coefficients le sont à 1% sauf la distance de la gare la plus proche, dont le coefficient est significatif à 5%. 

Augmenter d'un point de pourcentage la proportion de cadre dans la commune augmente de 0.07% le nombre de voyageur dans une gare. Le coefficient ne peut pas être interprété comme un effect causal dû à l'endogénéité du modèle. Il peut y avoir une causalité inverse (les gares attirent les personnes avec un diplôme), mais cette endogénéité ne sera pas traitée ici.

# 3. Conclusion

Nous avons montrer qu'il existe un lien entre le pourcentage de catégories socio-professionnels et le nombre de voyageurs dans les communes. Cela ne suffit pas à en déduire un lien causal à cause de l'endogénéité du model. En revanche, cela nous donne un modèle de prédiction. 